In [7]:
# Data Extraction
import pandas as pd
import psycopg2

# Establish connection to the database
conn = psycopg2.connect(
    host="",
    # port =,
    database="",
    user="",
    password=""
)

# Query for home games from Offensive_Stats and Defensive_Stats
home_offense_query = "SELECT date, location, result, total_yards FROM Offensive_Stats WHERE location = 'H'"
home_offense_df = pd.read_sql(home_offense_query, conn)

home_defense_query = "SELECT date, location, opponent_total_yards FROM Defensive_Stats WHERE location = 'H'"
home_defense_df = pd.read_sql(home_defense_query, conn)

# Query for away games from Offensive_Stats and Defensive_Stats
away_offense_query = "SELECT date, location, result, total_yards FROM Offensive_Stats WHERE location = '@'"
away_offense_df = pd.read_sql(away_offense_query, conn)

away_defense_query = "SELECT date, location, opponent_total_yards FROM Defensive_Stats WHERE location = '@'"
away_defense_df = pd.read_sql(away_defense_query, conn)

# Close the connection for now
conn.close()

# Merge the offensive and defensive data on date and location
home_df = pd.merge(home_offense_df, home_defense_df, on=['date', 'location'], how='inner')
away_df = pd.merge(away_offense_df, away_defense_df, on=['date', 'location'], how='inner')

print("Home Data Sample:")
print(home_df.head())

print("Away Data Sample:")
print(away_df.head())




Home Data Sample:
         date location     result  total_yards  opponent_total_yards
0  2023-08-26        H  W (56-28)          501                   396
1  2023-09-02        H  W (66-14)          668                   345
2  2023-09-02        H  W (66-14)          668                   360
3  2023-09-02        H  W (66-14)          668                   200
4  2023-09-02        H  W (66-14)          668                   402
Away Data Sample:
         date location     result  total_yards  opponent_total_yards
0  2023-09-23        @  W (42-28)          535                   219
1  2023-09-23        @  W (42-28)          535                   353
2  2023-09-30        @  W (48-41)          498                   564
3  2023-09-30        @  W (48-41)          498                   222
4  2023-09-30        @  W (48-41)          498                   342


/var/folders/d0/gj8tfnxn3jd2gjx88tk4mp180000gn/T/ipykernel_25588/3292354984.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  home_offense_df = pd.read_sql(home_offense_query, conn)
/var/folders/d0/gj8tfnxn3jd2gjx88tk4mp180000gn/T/ipykernel_25588/3292354984.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  home_defense_df = pd.read_sql(home_defense_query, conn)
/var/folders/d0/gj8tfnxn3jd2gjx88tk4mp180000gn/T/ipykernel_25588/3292354984.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  away_offense_df = pd.read_sql(

In [8]:
# Comparative Analysis

# Function to extract points from the result string
def extract_points(result):
    # Assumes format 'W (XX-YY)' or 'L (XX-YY)'
    points_scored = int(result.split('(')[-1].split('-')[0])
    points_allowed = int(result.split('-')[1].split(')')[0])
    return points_scored, points_allowed

# Apply function to calculate points scored and allowed
home_df['points_scored'], home_df['points_allowed'] = zip(*home_df['result'].apply(extract_points))
away_df['points_scored'], away_df['points_allowed'] = zip(*away_df['result'].apply(extract_points))

# Calculate totals for home games
home_totals = {
    'total_points_scored': home_df['points_scored'].sum(),
    'total_points_allowed': home_df['points_allowed'].sum(),
    'total_yards_for': home_df['total_yards'].sum(),
    'total_yards_against': home_df['opponent_total_yards'].sum(),
    'win_count': (home_df['result'].str.contains('W')).sum(),
    'loss_count': (home_df['result'].str.contains('L')).sum()
}

# Calculate totals for away games
away_totals = {
    'total_points_scored': away_df['points_scored'].sum(),
    'total_points_allowed': away_df['points_allowed'].sum(),
    'total_yards_for': away_df['total_yards'].sum(),
    'total_yards_against': away_df['opponent_total_yards'].sum(),
    'win_count': (away_df['result'].str.contains('W')).sum(),
    'loss_count': (away_df['result'].str.contains('L')).sum()
}

print("Home Totals:", home_totals)
print("Away Totals:", away_totals)



Home Totals: {'total_points_scored': 23435, 'total_points_allowed': 13912, 'total_yards_for': 289130, 'total_yards_against': 223684, 'win_count': 472, 'loss_count': 137}
Away Totals: {'total_points_scored': 13164, 'total_points_allowed': 11580, 'total_yards_for': 182776, 'total_yards_against': 168174, 'win_count': 233, 'loss_count': 184}
